---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
jupyter: python3
---


# Pytorch Lightning: Hyperparameter Tuning with Diabetes Data


In [ ]:
#| echo: false
#| label: 100_imports
import numpy as np
import os
from math import inf
import numpy as np
import warnings
if not os.path.exists('./figures'):
    os.makedirs('./figures')
warnings.filterwarnings("ignore")

In this tutorial, we will show how `spotPython` can be integrated into the `PyTorch` Lightning
training workflow for a regression task.

## The Diabetes Data Set {#sec-the-diabetes-data-set-100}

This chapter describes the hyperparameter tuning of a `PyTorch Lightning` network on the Diabetes data set. This is a PyTorch Dataset for regression. A toy data set from scikit-learn. Ten baseline variables, age, sex, body mass index, average blood pressure, and six blood serum measurements were obtained for each of n = 442 diabetes patients,  as well as the response of interest, a quantitative measure of disease progression one year after baseline.

### Data Exploration of the sklearn Diabetes Data Set


In [ ]:
from sklearn.datasets import load_diabetes
from spotPython.plot.xy import plot_y_vs_X
data = load_diabetes()
X, y = data.data, data.target
plot_y_vs_X(X, y, nrows=5, ncols=2, figsize=(20, 15))

* Each of these 10 feature variables have been mean centered and scaled by the standard deviation times the square root of n_samples (i.e. the sum of squares of each column totals 1).

* HDL (high-density lipoprotein) cholesterol, sometimes called “good” cholesterol, absorbs cholesterol in the blood and carries it back to the liver.
* The liver then flushes it from the body.
* High levels of HDL cholesterol can lower your risk for heart disease and stroke.

### Generating the PyTorch data set


In [ ]:
from spotPython.data.diabetes import Diabetes
data_set = Diabetes()
print(len(data_set))
print(data_set.names)

## Setup{#sec-setup-24}

### General Experiment Setup{#sec-general-experiment-setup-24}

To keep track of the different experiments, we use a `PREFIX` for the experiment name. The `PREFIX` is used to create a unique experiment name. The `PREFIX` is also used to create a unique TensorBoard folder, which is used to store the TensorBoard log files.

`spotpython` allows the specification of two different types of stopping criteria: first, the number of function evaluations (`fun_evals`), and second, the maximum run time in seconds (`max_time`). Here, we will set the number of function evaluations to infinity and the maximum run time to one minute.

Furthermore, we set the initial design size (`init_size`) to 10. The initial design is used to train the surrogate model. The surrogate model is used to predict the performance of the hyperparameter configurations. The initial design is also used to train the first model. Since the `init_size` belongs to the experimental design, it is set in the `design_control` dictionary, see [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/utils/init/#spotPython.utils.init.design_control_init).

`max_time` is set to one minute for demonstration purposes and `init_size` is set to 10 for demonstration purposes. For real experiments, these values should be increased.
Note,  the total run time may exceed the specified `max_time`, because the initial design is always evaluated, even if this takes longer than `max_time`.

The following parameters are used to specify the general experiment setup:


In [ ]:
#| label: 024_sum_exp
PREFIX = "100"
fun_evals = inf
max_time = 1
init_size = 10

### Data Preprocessing {#sec-preprocessing-100}

Preprocessing is handled by `Lightning` and `PyTorch`. It is described in the [LIGHTNINGDATAMODULE](https://lightning.ai/docs/pytorch/stable/data/datamodule.html) documentation. Here you can find information about the `transforms` methods.

###  Data Splitting

The data splitting is handled by `Lightning`.

### Loss Function {#sec-loss-function-100}

The loss function is specified in the configurable network class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/regression/nn_linear_regressor.py).
We will use the mean squared error (MSE) as the loss function. i.e., from `torchmetrics` the function ` mean_squared_error`, see [[SOURCE]](https://github.com/Lightning-AI/torchmetrics/blob/master/src/torchmetrics/functional/regression/mse.py).
The user can specify any of the loss functions from `torchmetrics` via the argument `_torchmetric` in the `fun_control` dictionary, see [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/utils/init/#spotPython.utils.init.fun_control_init).

A detailed description of the loss functions (torchmetrics) is presented in @sec-torchmetrics-100.


### The Objective Function `fun` {#sec-the-objective-function-31}

The objective function `fun` from the class `HyperLight` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/fun/hyperlight.py) is selected next. It implements an interface from `PyTorch`'s training, validation, and testing methods to `spotPython`.


In [ ]:
from spotPython.fun.hyperlight import HyperLight
fun = HyperLight().fun

### Model Setup

By using `core_model_name = "light.regression.NNLinearRegressor"`, the `spotPython` model class `NetLightRegression` [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/light/regression/netlightregression/) from the `light.regression` module is selected.
For a given `core_model_name`, the corresponding hyperparameters are automatically loaded from the associated dictionary, which is stored as a JSON file. The JSON file contains hyperparameter type information, names, and bounds. For `spotPython` models, the hyperparameters are stored in the `LightHyperDict`, see [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/hyperdict/light_hyper_dict.json)
Alternatively, you can load a local hyper_dict.
The `hyperdict`  uses the default hyperparameter settings. These can be modified as described in @sec-modifying-hyperparameter-levels.

The following parameters are used for the model setup:


In [ ]:
#| label: 100_sum_model-setup
from spotPython.hyperdict.light_hyper_dict import LightHyperDict
core_model_name = "light.regression.NNLinearRegressor"
hyperdict = LightHyperDict
_L_in=10
_L_out=1

### Surrogate Model Setup

The default surrogate model is the `Kriging` model, see [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/build/kriging/). We specify `noise` as `True` to include noise in the model. An `anisotropic` kernel is used, which allows different length scales for each dimension, by setting `n_theta = 2`. Furthermore, the interval for the `Lambda` value is set to `[1e-3, 1e2]`.

These parameters are set in the `surrogate_control` dictionary and therefore passed  to the `surrogate_control_init` function [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/utils/init/#spotPython.utils.init.surrogate_control_init).


In [ ]:
#| label: 100_surrogate_control_setup
noise = True
n_theta = 2
min_Lambda = 1e-3
max_Lambda = 10

### Summary: Setting up the Experiment {#sec-summary-setting-up-the-experiment-24}

At this stage, all required information is available to set up the dictionaries for the hyperparameter tuning.
The class `Spot` [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/spot/spot.py) is the hyperparameter tuning workhorse. It is initialized with the following parameters, which were specified above.

* `fun`: the objective function
* `fun_control`: the dictionary with the control parameters for the objective function
* `design_control`: the dictionary with the control parameters for the experimental design
* `surrogate_control`: the dictionary with the control parameters for the surrogate model
* `optimizer_control`: the dictionary with the control parameters for the optimizer

`spotpython` allows maximum flexibility in the definition of the hyperparameter tuning setup. Alternative surrogate models, optimizers, and experimental designs can be used. Thus, interfaces for the `surrogate` model, experimental `design`, and `optimizer` are provided. The default surrogate model is the kriging model, the default optimizer is the differential evolution, and default experimental design is the Latin hypercube design.

The information discussed so far can be summarized as follows:


In [ ]:
#| label: 100_summary_control
from spotPython.data.diabetes import Diabetes
from spotPython.hyperdict.light_hyper_dict import LightHyperDict
from spotPython.fun.hyperlight import HyperLight
from spotPython.utils.init import (fun_control_init, surrogate_control_init)


fun = HyperLight().fun

fun_control = fun_control_init(
    PREFIX="100",
    fun_evals=inf,
    max_time=1,
    data_set = Diabetes(),
    core_model_name="light.regression.NNLinearRegressor",
    hyperdict=LightHyperDict,
    _L_in=10,
    _L_out=1,
    _torchmetric="mean_squared_error",
    TENSORBOARD_CLEAN=True,
   )
surrogate_control = surrogate_control_init(n_theta=2, noise=True, min_Lambda=1e-3, max_Lambda=10)

### Checking the Experimental Design


In [ ]:
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control))

In [ ]:
from spotPython.hyperparameters.values import set_hyperparameter
set_hyperparameter(fun_control, "initialization", ["Default"])
print(gen_design_table(fun_control))

:::

## Tensorboard {#sec-tensorboard-100}

The textual output shown in the console (or code cell) can be visualized with Tensorboard.



```{raw}
tensorboard --logdir="runs/"
```


Further information can be found in the [PyTorch Lightning documentation](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.loggers.tensorboard.html) for Tensorboard.

## Running the Hyperparameter Tuning Experiment


In [ ]:
#| label: 100_spot_run
from spotPython.spot import spot
spot_tuner = spot.Spot(
    fun=fun,
    fun_control=fun_control,
    surrogate_control=surrogate_control,
)
res = spot_tuner.run()

## Using the `spotgui`

The `spotgui` [[github]](https://github.com/sequential-parameter-optimization/spotGUI) provides a convenient way to interact with the hyperparameter tuning process.
To obtain the settings from @sec-summary-setting-up-the-experiment-24, the `spotgui` can be started as shown in @fig-spotgui.

![spotgui](./figures_static/024_gui.png){width=100% #fig-spotgui}


## Results

### Progress Plot


After the hyperparameter tuning run is finished, the progress of the hyperparameter tuning can be visualized with `spotpython`'s method `plot_progress`. The black points represent the performace values (score or metric) of  hyperparameter configurations from the initial design, whereas the red points represents the  hyperparameter configurations found by the surrogate model based optimization.


In [ ]:
spot_tuner.plot_progress(log_y=True, filename=None)

### Tuned Hyperparameters and Their Importance

Results can be printed in tabular form.


In [ ]:
from spotPython.utils.eda import gen_design_table
print(gen_design_table(fun_control=fun_control, spot=spot_tuner))

A histogram can be used to visualize the most important hyperparameters.


In [ ]:
spot_tuner.plot_importance(threshold=10.0)

### Detailed Hyperparameter Plots


In [ ]:
#| label: 024_plot_important_hyperparameter_contour
spot_tuner.plot_important_hyperparameter_contour(max_imp=3)

### Parallel Coordinates Plots


In [ ]:
#| label: 024_parallel_plot
spot_tuner.parallel_plot()

## Load the saved experiment and get the hyperparameters (tuned architecture)


In [ ]:
from spotPython.utils.file import get_experiment_from_PREFIX
config = get_experiment_from_PREFIX("100")["config"]
config

### Cross Validation With Lightning

* The `KFold` class from `sklearn.model_selection` is used to generate the folds for cross-validation.
* These mechanism is used to generate the folds for the final evaluation of the model.
* The `CrossValidationDataModule` class [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/data/lightcrossvalidationdatamodule.py) is used to generate the folds for the hyperparameter tuning process.
* It is called from the `cv_model` function [[SOURCE]](https://github.com/sequential-parameter-optimization/spotPython/blob/main/src/spotPython/light/cvmodel.py).


In [ ]:
from spotPython.light.cvmodel import cv_model
from spotPython.hyperparameters.values import set_control_key_value
set_control_key_value(control_dict=fun_control,
                        key="k_folds",
                        value=2,
                        replace=True)
set_control_key_value(control_dict=fun_control,
                        key="test_size",
                        value=0.6,
                        replace=True)
cv_model(config, fun_control)

## Test on the full data set


In [ ]:
from spotPython.light.testmodel import test_model
test_model(config, fun_control)

## Load the last model


In [ ]:
from spotPython.light.loadmodel import load_light_from_checkpoint
model_loaded = load_light_from_checkpoint(config, fun_control)

In [ ]:
from spotPython.utils.init import get_feature_names
get_feature_names(fun_control)

### Integrated Gradients


In [ ]:
from spotPython.plot.xai import get_attributions, plot_attributions
df = get_attributions(spot_tuner, fun_control, attr_method="IntegratedGradients")
print(df)
plot_attributions(df)

### Deep Lift


In [ ]:
df = get_attributions(spot_tuner, fun_control, attr_method="DeepLift")
print(df)
plot_attributions(df,  attr_method="DeepLift")

### Feature Ablation


In [ ]:
df = get_attributions(spot_tuner, fun_control, attr_method="FeatureAblation")
print(df)
plot_attributions(df, attr_method="FeatureAblation")

## Visualizing the Activations, Weights, and Gradients

In neural networks, activations, weights, and gradients are fundamental concepts that play different.

1. Activations:

   Activations refer to the outputs of neurons after applying an activation function. In neural networks, the input passes through each neuron of the network layers, where each unit calculates a weighted sum of its inputs and then applies a non-linear activation function (such as ReLU, Sigmoid, or Tanh). These activation functions help introduce non-linearity into the model, enabling the neural network to learn complex relationships between the input data and the predictions. In short, activations are the outputs that are forwarded by the neurons after applying the activation function.

2. Weights:

   Weights are parameters within a neural network that control the strength of the connection between two neurons in successive layers. They are adjusted during the training process to enable the neural network to perform the desired task as well as possible. Each input is multiplied by a weight, and the neural network learns by adjusting these weights based on the error between the predictions and the actual values. Adjusting the weights allows the network to recognize patterns and relationships in the input data and use them for predictions or classifications.

3. Gradients:

   In the context of machine learning and specifically in neural networks, gradients are a measure of the rate of change or the slope of the loss function (a function that measures how well the network performs in predicting the desired output) with respect to the weights. During the training process, the goal is to minimize the value of the loss function to improve the model’s performance. The gradients indicate the direction and size of the steps that need to be taken to adjust the weights in a way that minimizes the loss (known as gradient descent). By repeatedly adjusting the weights in the opposite direction of the gradient, the network can be effectively trained to improve its prediction accuracy.

::: {.callout-note}
### Reference:

* The following code is based on [[PyTorch Lightning TUTORIAL 2: ACTIVATION FUNCTIONS]](https://lightning.ai/docs/pytorch/stable/notebooks/course_UvA-DL/02-activation-functions.html), Author: Phillip Lippe, License: [[CC BY-SA]](https://creativecommons.org/licenses/by-sa/3.0/), Generated: 2023-03-15T09:52:39.179933.

:::

After we have trained the models, we can look at the actual activation values that find inside the model. For instance, how many neurons are set to zero in ReLU? Where do we find most values in Tanh? To answer these questions, we can write a simple function which takes a trained model, applies it to a batch of images, and plots the histogram of the activations inside the network:


In [ ]:
from spotPython.plot.xai import (get_activations, get_gradients, get_weights, plot_nn_values_hist, plot_nn_values_scatter, visualize_weights, visualize_gradients, visualize_activations, visualize_activations_distributions, visualize_gradient_distributions, visualize_weights_distributions)
import pprint

In [ ]:
from spotPython.utils.file import get_experiment_from_PREFIX
get_experiment_from_PREFIX("100", return_dict=False)
pprint.pprint(config)
batch_size = config["batch_size"]
print(batch_size)

In [ ]:
from spotPython.light.loadmodel import load_light_from_checkpoint
model_loaded = load_light_from_checkpoint(config, fun_control)
model = model_loaded.to("cpu")

### Weights


In [ ]:
weights, index = get_weights(model, return_index=True)
print(index)

In [ ]:
visualize_weights(model, absolute=True, cmap="gray", figsize=(6, 6))

In [ ]:
visualize_weights_distributions(model, color=f"C{0}")

### Activations


In [ ]:
activations = get_activations(model, fun_control=fun_control, batch_size=batch_size, device = "cpu")

In [ ]:
visualize_activations(model, fun_control=fun_control, batch_size=batch_size, device = "cpu", cmap="BlueWhiteRed", absolute=False)

* Absolute values of the activations are plotted:


In [ ]:
visualize_activations(model, fun_control=fun_control, batch_size=batch_size, device = "cpu", absolute=True)

In [ ]:
visualize_activations_distributions(net=model, fun_control=fun_control, batch_size=batch_size, device="cpu", color="C0", columns=2)

### Gradients


In [ ]:
gradients = get_gradients(model, fun_control, batch_size, device="cpu")

In [ ]:
visualize_gradients(model, fun_control, batch_size, absolute=True, cmap="BlueWhiteRed", figsize=(6, 6))

In [ ]:
visualize_gradient_distributions(model, fun_control, batch_size=batch_size, color=f"C{0}")

## Layer Conductance


In [ ]:
from spotPython.plot.xai import get_weights_conductance_last_layer, plot_conductance_last_layer
w, c = get_weights_conductance_last_layer(spot_tuner, fun_control)
plot_conductance_last_layer(w,c)

## Details of the Implementation

### Loss Function {#sec-loss-function-100}


The parameter `_torchmetric` is used to specify the metric. Any metric from the `torchmetrics` library can be used.
Currently, the following metrics are provided, see [[LINK]](https://lightning.ai/docs/torchmetrics/stable/all-metrics.html):

* `concordance_corrcoef`
* `cosine_similarity`
* `critical_success_index`
* `explained_variance`
* `kendall_rank_corrcoef`
* `kl_divergence`
* `log_cosh_error`
* `mean_squared_log_error`
* `mean_absolute_error`
* `mean_squared_error`
* `pearson_corrcoef`
* `mean_absolute_percentage_error`
* `mean_absolute_percentage_error`
* `minkowski_distance`
* `r2_score`
* `relative_squared_error`
* `spearman_corrcoef`
* `symmetric_mean_absolute_percentage_error`
* `tweedie_deviance_score`
* `weighted_mean_absolute_percentage_error`



 The metric is used in the training, validation, and testing steps of the model.


`NNLightRegressor` implements a loss function as follows:

```python
    def _calculate_loss(self, batch):
        """
        Calculate the loss for the given batch.

        Args:
            batch (tuple): A tuple containing a batch of input data and labels.
            mode (str, optional): The mode of the model. Defaults to "train".

        Returns:
            torch.Tensor: A tensor containing the loss for this batch.

        """
        x, y = batch
        y = y.view(len(y), 1)
        y_hat = self(x)
        loss = self.metric(y_hat, y)
        return loss
```

The loss function is used in the training, validation, and testing steps of the model.

### Data Handling {#sec-data-handling-100}

As shown below, a DataLoader from `torch.utils.data` can be used to check the data. `spotPython` implements a `DataModule` class, which is used to load the data. The `DataModule` class is called from the `fun` function. The `DataModule` class is used to load the data and to split the data into training, validation, and test sets.


In [ ]:
from spotPython.data.diabetes import Diabetes
import torch
data_set = Diabetes(target_type=torch.float)
print(len(data_set))
# Set batch size for DataLoader
batch_size = 5
# Create DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(data_set, batch_size=batch_size, shuffle=False)

# Iterate over the data in the DataLoader
for batch in dataloader:
    inputs, targets = batch
    print(f"Batch Size: {inputs.size(0)}")
    print(f"Inputs Shape: {inputs.shape}")
    print(f"Targets Shape: {targets.shape}")
    print("---------------")
    print(f"Inputs: {inputs}")
    print(f"Targets: {targets}")
    break

Information related to the data is set in the `fun_control` dictionary, see [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/utils/init/#spotPython.utils.init.fun_control_init).
Specifically, the following parameters are set:

* `data_set`: the data set
* `num_workers`: the number of workers
* `test_size`: the size of the test set
* `test_seed`: the seed for the test set


In [ ]:
from spotPython.utils.init import fun_control_init
from spotPython.data.diabetes import Diabetes
data_set = Diabetes(target_type=torch.float)
fun_control = fun_control_init(
    data_set=data_set,
    device="cpu",
    enable_progress_bar=False,
    num_workers=0,
    show_progress=True,
    test_size=0.4,
    test_seed=42,
    )

### `spotPython`'s  `LightDataModule` Class

The values from the `fun_control` dictionary are used to set up the `LightDataModule` class, see [[SOURCE]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotPython/data/lightdatamodule/).
The parameter `batch_size` is a hyperparameter that can be tuned and therefore not set in the `fun_control` dictionary.


In [ ]:
from spotPython.data.lightdatamodule import LightDataModule
dm = LightDataModule(
    dataset=fun_control["data_set"],
    batch_size=5,
    num_workers=fun_control["num_workers"],
    test_size=fun_control["test_size"],
    test_seed=fun_control["test_seed"],
)
dm.setup()
print(f"train_model(): Test set size: {len(dm.data_test)}")
print(f"train_model(): Train set size: {len(dm.data_train)}")

The steps described above are handled by the `LightDataModule` class. This class is used to create the data loaders for the training, validation, and test sets. The `LightDataModule` class is part of the `spotPython` package.
The `LightDataModule` class provides the following methods:

* `prepare_data()`: This method is used to prepare the data set.
* `setup()`: This method is used to create the data loaders for the training, validation, and test sets.
* `train_dataloader()`: This method is used to return the data loader for the training set.
* `val_dataloader()`: This method is used to return the data loader for the validation set.
* `test_dataloader()`: This method is used to return the data loader for the test set.
* `predict_dataloader()`: This method is used to return the data loader for the prediction set.

Details of the implementation can be found in @sec-hpt-pytorch-lightning-data-30.

## HyperLight: Interfacing the Optimizer `spotPython`  with the Data, the Loss Function, and the Model

### The `HyperLight` Class Method `fun()`

The class `Hyperlight` implements the method `fun()`, which passes the hyperparameter tuning parametrizations to the neural network and the configurations to the DataModule. The `fun()` method passes one configuration to the `train_model()` method, which trains the model and returns the performance value. The performance value is then returned to the optimizer.

```python
df_eval = train_model(config, fun_control)
```

### The Method `train_model`

`train_model` trains a model using the given configuration and function control parameters.
It extracts the information from the configuration and the function control parameters and trains the model using the `Lightning` framework.
A simplified version of the `train_model` method is shown below:


In [ ]:
def train_model(config: dict, fun_control: dict, timestamp: bool = True) -> float:
    _L_in = fun_control["_L_in"]
    _L_out = fun_control["_L_out"]
    _torchmetric = fun_control["_torchmetric"]
    model = fun_control["core_model"](**config, _L_in=_L_in, _L_out=_L_out, _torchmetric=_torchmetric)
    dm = LightDataModule(
        dataset=fun_control["data_set"],
        batch_size=config["batch_size"],
        num_workers=fun_control["num_workers"],
        test_size=fun_control["test_size"],
        test_seed=fun_control["test_seed"],
        scaler=fun_control["scaler"],
    )
    trainer = L.Trainer()
    trainer.fit(model=model, datamodule=dm)
    # Test best model on validation set
    result = trainer.validate(model=model, datamodule=dm)
    result = result[0]
    return result["val_loss"]

## Summary of the Lightning Training Process in `spotPython`

The following code exemplifies the workflow and shows how to combine the elements discussed so far:

* First, results from the tuning experiment ("100") are loaded.
* A `LightDataModule` object is created and the `setup()` method is called.
* Then, the `Trainer` is initialized.
* Load the NN model from the last checkpoint.
* Next, the `fit()` method is called to train the model.
* Finally, the `validate()` method is called to validate the model. The `validate()` method returns the validation loss.


In [ ]:
# Load results from the experiment 100
from spotPython.utils.file import get_experiment_from_PREFIX
config, fun_control, design_control, surrogate_control, optimizer_control = get_experiment_from_PREFIX("100",return_dict=False)

In [ ]:
# Create a LightDataModule object
from spotPython.data.lightdatamodule import LightDataModule
dm = LightDataModule(
    dataset=fun_control["data_set"],
    batch_size=config["batch_size"],
    num_workers=fun_control["num_workers"],
    test_size=fun_control["test_size"],
    test_seed=fun_control["test_seed"],
)
dm.setup()

In [ ]:
# Initialize the Trainer
import lightning as L
trainer = L.Trainer(enable_progress_bar=False)

In [ ]:
# Load the model from the checkpoint
from spotPython.light.loadmodel import load_light_from_checkpoint
model_loaded = load_light_from_checkpoint(config, fun_control)
model = model_loaded.to("cpu")

In [ ]:
# Fit (train) the model
trainer.fit(model=model, datamodule=dm)

In [ ]:
# Validation: Test best model on validation set
result = trainer.validate(model=model, datamodule=dm)
# unlist the result (from a list of one dict)
result = result[0]
print(result["val_loss"])

## Loading a User Specified Data Set

Using a user-specified data set is straightforward. The user simply needs to provide a data set and loads is as a  `spotPython`  `CVSDataset()` class by specifying the path, filename, and target column as follows:


In [ ]:
from spotPython.data.csvdataset import CSVDataset
import torch
data_set = CSVDataset(directory="./userData/",
                     filename="data.csv",
                     target_column='target',
                     feature_type=torch.float32,
                     target_type=torch.float32,
                     rmNA=True)
print(len(data_set))
# Set batch size for DataLoader
batch_size = 5
# Create DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(data_set, batch_size=batch_size, shuffle=False)

# Iterate over the data in the DataLoader
for batch in dataloader:
    inputs, targets = batch
    print(f"Batch Size: {inputs.size(0)}")
    print(f"Inputs Shape: {inputs.shape}")
    print(f"Targets Shape: {targets.shape}")
    print("---------------")
    print(f"Inputs: {inputs}")
    print(f"Targets: {targets}")
    break

## Using a User Specified Model

As templates, we provide the following three files that allow the user to specify a model in the `/userModel` directory:
    * `my_regressor.py`
    * `my_hyperdict.json`
    * `my_hyperdict.py`

The `my_regressor.py` file contains the model class, which is a subclass of `nn.Module`. The `my_hyperdict.json` file contains the hyperparameter settings as a dictionary, which are loaded via the `my_hyperdict.py` file.


In [ ]:
from spotPython.hyperparameters.values import add_core_model_to_fun_control
import sys
sys.path.insert(0, './userModel')
import my_regressor
import my_hyper_dict
add_core_model_to_fun_control(fun_control=fun_control,
                              core_model=my_regressor.MyRegressor,
                              hyper_dict=my_hyper_dict.MyHyperDict)